In [16]:
from pathlib import Path
import json
import re
from tqdm import tqdm

settings = json.loads(Path('../../../settings.json').read_text())preprocessed_data_path = Path(settings['preprocessed_data_path'])data_path = Path('.').resolve()data_name = data_path.namesource_name = data_path.parent.namepreprocessed_dir = preprocessed_data_path/source_name/data_namesource_data_dir = preprocessed_dir/'preprocessed'
splits = ['train', 'valid']
tasks = ['의도 식별', 'LM(한국어)', 'dialog']
task_data_dir = preprocessed_dir/'preprocessed_task'
task_data_dir.mkdir(exist_ok=True)
for task in tasks:
    task_path = task_data_dir/task
    task_path.mkdir(exist_ok=True)
    
#### prepare for task preprocess



#### prepare for task preprocess end

#### task preprocess
for split in splits:
    source_data_dir_split = source_data_dir/split
    task_files = [(task_data_dir/task/f'{split}.jsonl').open('w', encoding='utf-8') for task in tasks]
    for source_data in tqdm(list(source_data_dir_split.iterdir()), desc=split):
        source_data = source_data.open()
        for line in source_data.readlines():
            line = json.loads(line)
            
            #### data preprocess
            first_spreaker = line['대화'][0]['화자']
            dialog = []
            for utt in line['대화']:
                new_utt = {
                    'speaker': utt['화자'],
                    'utterance': utt['고객질문(요청)']+utt['상담사질문(요청)']+utt['고객답변']+utt['상담사답변'],
                    'intent': [utt['고객의도']+utt['상담사의도'], ('질문' if utt["QA"]=='Q' else '답변')]
                }
                new_utt['intent'] = '/'.join([i for i in new_utt['intent'] if i])
                dialog.append(new_utt)
            #### data preprocess end 


            #### LM(한국어)
            data = {'text': None}
            ## preprocess data from line
            text = '##dialog info : '+ ', '.join([
                line['도메인'],
                line['카테고리'],
            ]) + '\n'
            speaker_id = {'고객': (first_spreaker!='고객') + 1, '상담사': (first_spreaker!='상담사')+1}
            text += '##dialog : '+'\n'.join([
                f'{speaker_id[utt['speaker']]} : ' + utt['utterance'] 
                for utt in dialog])
            data['text'] = text
            ## preprocess data from line end
            task_files[1].write(json.dumps(data, ensure_ascii=False)+'\n')
            # print(text)
            #### LM(한국어) end


            #### dialog
            data = {'text': {}}
            ## preprocess data from line
            data['text']['도메인'] = line['도메인']
            data['text']['카테고리'] = line['카테고리']
            data['text']['dialog'] = dialog
            ## preprocess data from line end
            task_files[2].write(json.dumps(data, ensure_ascii=False)+'\n')
            # print(json.dumps(data, indent=4, ensure_ascii=False))
            #### dialog end
            
            
            #### 의도 식별
            data = {'input': {}, 'output': None}
            ## preprocess data from line
            data['input']['도메인'] = line['도메인']
            data['input']['카테고리'] = line['카테고리']
            data['input']['대화'] = dialog
            intents = [utt.pop('intent') for utt in dialog]
            data['output'] = {'intent' : intents}
            ## preprocess data from line end
            task_files[0].write(json.dumps(data, ensure_ascii=False)+'\n')
            # print(json.dumps(data, indent=4, ensure_ascii=False))
            #### 의도 식별 end
            
            
    #         break
    #     break
    # break

    for path in task_files:
        path.close()      
        
#### task preprocess end      

valid: 100%|██████████| 25/25 [00:01<00:00, 19.47it/s]
